# Librerias

In [2]:
import pandas as pd
import requests
import json 
import folium 
import os
from datetime import datetime, timedelta

# USGS
## API Documentation - Earthquake Catalog
### URL
# Geophysics Institutions Throughout the World
- https://www.usgs.gov/programs/earthquake-hazards/science/geophysics-institutions-throughout-world
- https://earthquake.usgs.gov/fdsnws/event/1/[METHOD[?PARAMETERS]]
 - metodo: query
 - Parámetros:
   - Ubicación: Las peticiones que utilicen tanto rectángulo como círculo devolverán la intersección, que puede estar vacía, utilízala con precaución.
      * Conterminous U.S. [24.6, 50] Latitude. [-125, -65] Longitude.

https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=1995-05-04&minlatitude=24.6&maxlatitude=50&minlongitude=-125&maxlongitude=-65&limit=19999

In [11]:
def sismo(url):
    # Establecer el tiempo de inicio desde hoy menos 20000 eventos
    endtime = datetime.today().strftime('%Y-%m-%dT%H:%M:%S')
    starttime = (datetime.today() - timedelta(days=20000)).strftime('%Y-%m-%dT%H:%M:%S')

    # Establecer los demás parámetros de solicitud
    params = {"format": "geojson", "starttime": starttime, "endtime": endtime, "minmagnitude": 2.5,
            "minlatitude": 24.6, "maxlatitude": 50, "minlongitude": -125, "maxlongitude": -65, "limit": 20000}    
    data = requests.get(url, params = params)
    data = json.loads(data.text)
    return data

url = r"https://earthquake.usgs.gov/fdsnws/event/1/query?"
dic_sismo = sismo(url)
count=0
for i in (dic_sismo["features"]):
    if i["properties"]['cdi']:
        count += 1
# Convertir los datos a un dataframe
df_sismo = pd.json_normalize(dic_sismo['features'])

In [12]:
folder_path = "../DASHBOARD/CSV_ORIGINAL/"
file_name = "raw_USA.csv"
file_path = os.path.join(folder_path, file_name)

df_sismo.to_csv(file_path, index=False)

In [13]:
df_sismo.head()

,type,id,properties.mag,properties.place,properties.time,properties.updated,properties.tz,properties.url,properties.detail,properties.felt,...,properties.types,properties.nst,properties.dmin,properties.rms,properties.gap,properties.magType,properties.type,properties.title,geometry.type,geometry.coordinates
0,Feature,us6000kcpy,3.80,"8 km ENE of San Mateo, New Mexico",1684371858365,1684387784364,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,11.0,...,",dyfi,origin,phase-data,shakemap,",35.0,0.98900,0.71,38.0,ml,earthquake,"M 3.8 - 8 km ENE of San Mateo, New Mexico",Point,"[-107.5538, 35.3639, 14.93]"
1,Feature,us6000kckk,2.80,"17 km S of Stonewall Gap, Colorado",1684344170329,1684354752040,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,...,",origin,phase-data,",29.0,0.15700,1.58,35.0,ml,earthquake,"M 2.8 - 17 km S of Stonewall Gap, Colorado",Point,"[-105.0247, 36.9979, 5]"
2,Feature,nc73889196,2.84,"42km W of Ferndale, CA",1684339850600,1684386299040,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,...,",nearby-cities,origin,phase-data,scitech-link,",30.0,0.48030,0.24,263.0,md,earthquake,"M 2.8 - 42km W of Ferndale, CA",Point,"[-124.7598343, 40.5231667, 1.67]"
3,Feature,nc73889176,3.07,"16km SSE of Tres Pinos, CA",1684335705260,1684368311806,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,4.0,...,",dyfi,focal-mechanism,nearby-cities,origin,pha...",66.0,0.05088,0.12,77.0,ml,earthquake,"M 3.1 - 16km SSE of Tres Pinos, CA",Point,"[-121.272, 36.6485, 3.75]"
4,Feature,nc73889151,2.54,"16km SSE of Tres Pinos, CA",1684333505270,1684357451488,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,...,",focal-mechanism,nearby-cities,origin,phase-da...",59.0,0.05078,0.11,105.0,md,earthquake,"M 2.5 - 16km SSE of Tres Pinos, CA",Point,"[-121.2738333, 36.6496667, 3.5]"


Borro las columnas que no necesito

In [14]:
df_sismo.columns

Index(['type', 'id', 'properties.mag', 'properties.place', 'properties.time',
       'properties.updated', 'properties.tz', 'properties.url',
       'properties.detail', 'properties.felt', 'properties.cdi',
       'properties.mmi', 'properties.alert', 'properties.status',
       'properties.tsunami', 'properties.sig', 'properties.net',
       'properties.code', 'properties.ids', 'properties.sources',
       'properties.types', 'properties.nst', 'properties.dmin',
       'properties.rms', 'properties.gap', 'properties.magType',
       'properties.type', 'properties.title', 'geometry.type',
       'geometry.coordinates'],
      dtype='object')

In [15]:
df_sismo.drop(columns=['type','properties.alert','properties.tz','properties.code','properties.url','properties.detail','properties.net'
                       ,'properties.status','properties.types','properties.sources','properties.magType','properties.type','geometry.type',
                       'properties.title'], axis=1, inplace=True)

Cambio el formato de la columna properties.time

In [16]:
df_sismo["properties.time"] = pd.to_datetime(df_sismo["properties.time"], unit="ms") 
df_sismo["properties.updated"] = pd.to_datetime(df_sismo["properties.updated"], unit="ms")

In [17]:
df_sismo['estado'] = df_sismo['properties.place'].str.split(',').str[1].str.strip()

df_sismo['properties.place'] = df_sismo['properties.place'].str.split(',', expand=True)[0]


In [18]:
df_sismo.head(5)

,id,properties.mag,properties.place,properties.time,properties.updated,properties.felt,properties.cdi,properties.mmi,properties.tsunami,properties.sig,properties.ids,properties.nst,properties.dmin,properties.rms,properties.gap,geometry.coordinates,estado
0,us6000kcpy,3.80,8 km ENE of San Mateo,2023-05-18 01:04:18.365,2023-05-18 05:29:44.364,11.0,4.1,3.573,0,227,",us6000kcpy,",35.0,0.98900,0.71,38.0,"[-107.5538, 35.3639, 14.93]",New Mexico
1,us6000kckk,2.80,17 km S of Stonewall Gap,2023-05-17 17:22:50.329,2023-05-17 20:19:12.040,NaN,NaN,NaN,0,121,",us6000kckk,",29.0,0.15700,1.58,35.0,"[-105.0247, 36.9979, 5]",Colorado
2,nc73889196,2.84,42km W of Ferndale,2023-05-17 16:10:50.600,2023-05-18 05:04:59.040,NaN,NaN,NaN,0,124,",nc73889196,us6000kck7,",30.0,0.48030,0.24,263.0,"[-124.7598343, 40.5231667, 1.67]",CA
3,nc73889176,3.07,16km SSE of Tres Pinos,2023-05-17 15:01:45.260,2023-05-18 00:05:11.806,4.0,2.0,NaN,0,146,",nc73889176,",66.0,0.05088,0.12,77.0,"[-121.272, 36.6485, 3.75]",CA
4,nc73889151,2.54,16km SSE of Tres Pinos,2023-05-17 14:25:05.270,2023-05-17 21:04:11.488,NaN,NaN,NaN,0,99,",nc73889151,",59.0,0.05078,0.11,105.0,"[-121.2738333, 36.6496667, 3.5]",CA


In [19]:
#Divido la columna geometry coordinates
df_sismo[['Longitud', 'Latitud', 'Profundidad']] = df_sismo['geometry.coordinates'].apply(lambda x: pd.Series(str(x).strip('[]').split(',')))

In [20]:
df_sismo.head(1)

,id,properties.mag,properties.place,properties.time,properties.updated,properties.felt,properties.cdi,properties.mmi,properties.tsunami,properties.sig,properties.ids,properties.nst,properties.dmin,properties.rms,properties.gap,geometry.coordinates,estado,Longitud,Latitud,Profundidad
0,us6000kcpy,3.8,8 km ENE of San Mateo,2023-05-18 01:04:18.365,2023-05-18 05:29:44.364,11.0,4.1,3.573,0,227,",us6000kcpy,",35.0,0.989,0.71,38.0,"[-107.5538, 35.3639, 14.93]",New Mexico,-107.5538,35.3639,14.93


In [21]:
df_sismo.drop("geometry.coordinates", axis=1, inplace=True) 

In [22]:
df_sismo.columns

Index(['id', 'properties.mag', 'properties.place', 'properties.time',
       'properties.updated', 'properties.felt', 'properties.cdi',
       'properties.mmi', 'properties.tsunami', 'properties.sig',
       'properties.ids', 'properties.nst', 'properties.dmin', 'properties.rms',
       'properties.gap', 'estado', 'Longitud', 'Latitud', 'Profundidad'],
      dtype='object')

properties.sig : un número que describe la importancia del evento. Los números más grandes indican un evento más significativo. Este valor se determina en función de una serie de factores, que incluyen: magnitud, MMI máximo, informes de sensación e impacto estimado.

properties.dmin Distancia horizontal desde el epicentro hasta la estación más cercana (en grados). 1 grado son aproximadamente 111,2 kilómetros. En general, cuanto menor sea este número, más confiable es la profundidad calculada del terremoto.

	properties.gap: La brecha azimutal más grande entre estaciones azimutalmente adyacentes (en grados). En general, cuanto más pequeño es este número, más confiable es la posición horizontal calculada del terremoto. Las ubicaciones de terremotos en las que la brecha azimutal supera los 180 grados suelen tener grandes incertidumbres de ubicación y profundidad.

In [23]:
df_sismo = df_sismo.rename(columns={"properties.mag": "Magnitud",
                                    'properties.place': "Place",
                                    'properties.felt': "Felt",
                                    "properties.time": "Primer Registro",
                                    'properties.rms':"RMS",
                                    "properties.updated": "Ultimo registro",
                                    "properties.tsunami": "Posibilidad tsunami",
                                    "properties.sig": "Importancia del evento",
                                    "properties.dmin": "Dist Horizontal epicentro",
                                    "properties.gap": "Brecha azimutal",
                                    'properties.type': "Tipo",
                                    'geometry.type': "Tipo_geometria",
                                    'properties.cdi': 'cdi',  
                                    'properties.mmi': 'mmi',  
                                    'properties.net': 'net',  
                                    'properties.code': 'code',
                                    'properties.ids': 'ids',   
                                    'properties.nst': 'nst',
                                    'properties.alert': 'alert'})
df_sismo.head(1)

,id,Magnitud,Place,Primer Registro,Ultimo registro,Felt,cdi,mmi,Posibilidad tsunami,Importancia del evento,ids,nst,Dist Horizontal epicentro,RMS,Brecha azimutal,estado,Longitud,Latitud,Profundidad
0,us6000kcpy,3.8,8 km ENE of San Mateo,2023-05-18 01:04:18.365,2023-05-18 05:29:44.364,11.0,4.1,3.573,0,227,",us6000kcpy,",35.0,0.989,0.71,38.0,New Mexico,-107.5538,35.3639,14.93


In [24]:
df_sismo['estado'].unique()

array(['New Mexico', 'Colorado', 'CA', 'Oklahoma', 'Texas', 'Mexico',
       'Arizona', 'Kansas', 'Montana', 'Idaho', 'Nevada', nan,
       'Washington', None, 'Tennessee', 'B.C.', 'Wyoming', 'Canada',
       'New Hampshire', 'New York', 'California', 'Utah', 'Ohio',
       'Nebraska', 'Oregon', 'Missouri', 'Virginia', 'Maine', 'Kentucky',
       'Arkansas', 'Minnesota', 'Mississippi', 'North Carolina', 'Quebec',
       'Louisiana', 'South Carolina', 'Georgia', 'Alabama',
       'South Dakota', 'Illinois', 'NV', 'United States', 'Florida',
       'Iowa', 'MT', 'Maryland', 'NM', 'Michigan', 'ID', 'UT', 'OK', 'TN',
       'Massachusetts', 'TX', 'Pennsylvania', 'New Jersey',
       'NV Earthquake', 'KS', 'West Virginia', 'WA', 'WY', 'MO',
       'Delaware', 'Indiana'], dtype=object)

## Normalizacion de la columna estado

In [25]:
df_sismo['estado'] = df_sismo['estado'].replace({'United States': "Sin Dato",
                                                 'Sandywoods Township': "Sin Dato",
                                                 "CA":"California",
                                                 "WA": "Washington",
                                                 "WY": "Wyoming",
                                                 "UT": "Utah",
                                                 "TN": "Tennessee",
                                                 "TX": "Texas",
                                                 "ID": "Idaho",
                                                 "KS": "Kansas",
                                                 "MO": "Misuri",
                                                 "MT": "Montana",
                                                 "NM": "Nuevo Mexico",
                                                 "Mexico": "New Mexico",
                                                 "NV": "Nevada",
                                                 "NV Earthquake":"Nevada",
                                                 "OK": "Oklahoma",
                                                 'MN': "Minnesota",
                                                 'CO': 'Colorado',
                                                 'AZ': 'Arizona',
                                                 'OR': 'Oregon',
                                                 'Son.': 'Sonora',
                                                 'CA Earthquake': 'California',
                                                 'California Earthquake': "California",
                                                 'NE':"Nebraska"})

In [26]:
df_sismo['estado'].unique()

array(['New Mexico', 'Colorado', 'California', 'Oklahoma', 'Texas',
       'Arizona', 'Kansas', 'Montana', 'Idaho', 'Nevada', nan,
       'Washington', None, 'Tennessee', 'B.C.', 'Wyoming', 'Canada',
       'New Hampshire', 'New York', 'Utah', 'Ohio', 'Nebraska', 'Oregon',
       'Missouri', 'Virginia', 'Maine', 'Kentucky', 'Arkansas',
       'Minnesota', 'Mississippi', 'North Carolina', 'Quebec',
       'Louisiana', 'South Carolina', 'Georgia', 'Alabama',
       'South Dakota', 'Illinois', 'Sin Dato', 'Florida', 'Iowa',
       'Maryland', 'Nuevo Mexico', 'Michigan', 'Massachusetts',
       'Pennsylvania', 'New Jersey', 'West Virginia', 'Misuri',
       'Delaware', 'Indiana'], dtype=object)

In [27]:
df_sismo.isnull().sum()

id                               0
Magnitud                         0
Place                           47
Primer Registro                  0
Ultimo registro                  0
Felt                         10128
cdi                          10128
mmi                          17240
Posibilidad tsunami              0
Importancia del evento           0
ids                              0
nst                           5555
Dist Horizontal epicentro      891
RMS                              2
Brecha azimutal                 18
estado                         384
Longitud                         0
Latitud                          0
Profundidad                      0
dtype: int64

In [28]:
df_sismo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   id                         20000 non-null  object        
 1   Magnitud                   20000 non-null  float64       
 2   Place                      19953 non-null  object        
 3   Primer Registro            20000 non-null  datetime64[ns]
 4   Ultimo registro            20000 non-null  datetime64[ns]
 5   Felt                       9872 non-null   float64       
 6   cdi                        9872 non-null   float64       
 7   mmi                        2760 non-null   float64       
 8   Posibilidad tsunami        20000 non-null  int64         
 9   Importancia del evento     20000 non-null  int64         
 10  ids                        20000 non-null  object        
 11  nst                        14445 non-null  float64       
 12  Dist

Cambio el tipo de dato de Latitud y Longitud a float para poder utilizarlo en el mapa.

In [29]:
df_sismo["Longitud"] = df_sismo["Longitud"].astype(float) 
df_sismo["Latitud"] = df_sismo["Latitud"].astype(float) 
df_sismo["Profundidad"] = df_sismo["Profundidad"].astype(float)

DATOS UTILIZADOS

Alert
  - Tipo de datos: Cadena
  - Valores típicos: “verde”, “amarillo”, “naranja”, “rojo”.
  - Descripción: El nivel de alerta de la escala de impacto de terremotos de PAGER.  


**cdi**
  - Tipo de datos: decimal
  - Descripción: La máxima intensidad reportada para el evento. Calculado por DYFI. S 
i bien generalmente se informa como un número romano, para los fines de esta API, se espera que la intensidad  
sea el equivalente decimal del número romano. Obtenga más información sobre la magnitud frente a la intensidad. 
 

Depth 
- Tipo de datos: decimal 
- Profundidad del evento en kilómetros.

DepthError
  - Tipo de datos: decimal
  - Valores típicos: [0, 100]- Descripción: Incertidumbre de la profundidad reportada del evento en kilómetros.
  - Información adicional: El error de profundidad, en km, definido como la mayor proyección de los tres errores principales en una línea vertical. 


Dmin 
  - Tipo de datos: decimal
  - Descripción :Distancia horizontal desde el epicentro hasta la estación más cercana (en grados). 1 grado son aproximadamente 111,2 kilómetros.  
En general, cuanto menor sea este número, más confiable es la profundidad calculada del terremoto.

Gap
  - Tipo de datos: decimal
  - Descripción: La mayor brecha azimutal entre estaciones azimutalmente adyacentes (en grados). En general, cuanto más pequeño es este número, más confiable es la posición horizontal calculada del terremoto. Las ubicaciones de terremotos en las que la brecha azimutal supera los 180 grados suelen tener grandes incertidumbres de ubicación y profundidad.

HorizontalError
  - Tipo de datos: decimal
  - Descripción: Incertidumbre de la ubicación reportada del evento en kilómetros.
  - Información adicional: El error de ubicación horizontal, en km, definido como la longitud de la mayor proyección de los tres errores principales en un plano horizontal.  
Los errores principales son los ejes principales del elipsoide de error y son mutuamente perpendiculares.  
Las incertidumbres horizontales y verticales en la ubicación de un evento varían desde aproximadamente 100 m en horizontal y 300 metros en vertical para los eventos mejor ubicados, aquellos en medio de redes de sismógrafos densamente espaciadas, hasta decenas de kilómetros para eventos globales en muchas partes del mundo.  
Informamos un valor "desconocido" si la red sísmica contribuyente no proporciona estimaciones de incertidumbre. 

Latitud
  - Tipo de datos: decimal
  - Descripción :Grados decimales de latitud. Valores negativos para latitudes del sur.
  - Información adicional: Un terremoto comienza a romperse en un hipocentro que está definido por una posición en la superficie de la tierra (epicentro) y una profundidad por debajo de este punto (profundidad focal). Proporcionamos las coordenadas del epicentro en unidades de latitud y longitud. La latitud es el número de grados al norte (N) o al sur (S) del ecuador y varía de 0 en el ecuador a 90 en los polos.


LocationSource 
  - Tipo de datos: Cadena
  - Descripción: la red que originalmente creó la ubicación informada de este evento.

Longitud
  - Tipo de datos: decimal
  - Descripción :Grados decimales de longitud. Valores negativos para longitudes occidentales.
  - Información adicional: La longitud varía de 0 en Greenwich a 180 y el E o W muestra la dirección desde Greenwich. Las coordenadas se dan en el marco de referencia WGS84. La incertidumbre de la posición del hipocentro varía desde aproximadamente 100 m en horizontal y 300 metros en vertical para los eventos mejor ubicados, aquellos en medio de redes de sismógrafos densamente espaciadas, hasta decenas de kilómetros para eventos globales en muchas partes del mundo. 


mag 
 
- Información adicional: La magnitud reportada es la que el Servicio Geológico de los EE. UU. considera oficial para este terremoto, y fue la mejor estimación disponible del tamaño del terremoto, en el momento en que se creó esta página.

Mag Error
  - Tipo de datos: decimal
  - Descripción: Incertidumbre de la magnitud reportada del evento. El error estándar estimado de la magnitud. La incertidumbre corresponde al tipo de magnitud específica que se informa y no tiene en cuenta las variaciones de magnitud y los sesgos entre diferentes escalas de magnitud. Informamos un valor "desconocido" si la red sísmica contribuyente no proporciona estimaciones de incertidumbre.  
  

magNST
  - Tipo de datos: entero
  - - Descripción: El número total de estaciones sísmicas utilizadas para calcular la magnitud de este terremoto.

mmi
  - Tipo de datos: decimal
  - Descripción: La intensidad instrumental máxima estimada para el evento. Calculado por ShakeMap. Si bien generalmente se informa como un número romano, para los fines de esta API, se espera que la intensidad sea el equivalente decimal del número romano. Obtenga más información sobre la magnitud frente a la intensidad.


Nph
  - Número de fases utilizadas: Cadena
  - Descripción: número de observaciones de tiempo de llegada P y S utilizadas para calcular la ubicación del hipocentro. Un mayor número de observaciones de tiempo de llegada generalmente da como resultado mejores ubicaciones de terremotos.

Nst
  - Tipo de datos: entero
  - Descripción: El número total de estaciones sísmicas utilizadas para determinar la ubicación del terremoto.
  - Información Adicional: Número de estaciones sísmicas que reportaron tiempos de llegada P y S para este sismo. Este número: puede ser mayor que el Número de fases utilizadas si se rechazan los tiempos de llegada porque la distancia a una estación sísmica excede la distancia máxima permitida o porque la observación del tiempo de llegada es inconsistente con la solución.

Place
  - Tipo de datos: Cadena
  - - Descripción: Descripción textual de la región geográfica nombrada cercana al evento. Puede ser el nombre de una ciudad o el nombre de una región

Rms 
  - Tipo de datos: decimal
  - Descripción: El residual de tiempo de viaje de la raíz cuadrada media (RMS), en segundos, usando todos los pesos. Este parámetro proporciona una medida del ajuste de los tiempos de llegada observados a los tiempos de llegada previstos para esta ubicación. Los números más pequeños reflejan un mejor ajuste de los datos. El valor depende de la precisión del modelo de velocidad utilizado para calcular la ubicación del terremoto, los pesos de calidad asignados a los datos de tiempo de llegada y el procedimiento utilizado para localizar el terremoto. 

Sig (para discutir)
  - Tipo de datos: entero
  - Descripción: Un número que describe cuán significativo es el evento. Los números más grandes indican un evento más significativo. Este valor se determina en función de una serie de factores, que incluyen: magnitud, MMI máximo, informes de sensación e impacto estimado.

time
  - Tipo de datos: entero largo
  - Información: Indicamos la fecha y hora en que el sismo inicia la ruptura, lo que se conoce como hora "origen". Tenga en cuenta que los grandes terremotos pueden continuar rompiéndose durante muchas decenas de segundos. Proporcionamos la hora en UTC (Tiempo Universal Coordinado). Los sismólogos usan UTC para evitar la confusión causada por las zonas horarias locales y el horario de verano. En las páginas de eventos individuales, también se proporcionan los tiempos en el epicentro y su hora local en función de la hora configurada en su computadora.


tsunami
  - Tipo de datos: entero
  - Descripción: esta bandera se establece en "1" para grandes eventos en regiones oceánicas y "0" en caso contrario. La existencia o el valor de esta bandera no indica si realmente existió o existirá un tsunami.


Tipy(para discutir)
  - Tipo de datos: Cadena
  - Descripción :Tipo de evento sísmico.

Tz
  - Tipo de datos: entero
  - Descripción: Desplazamiento de zona horaria desde UTC en minutos en el epicentro del evento.

Updated
  - Tipo de datos: entero largo
  - Descripción: hora en que se actualizó el evento por última vez. Los tiempos se informan en milisegundos desde la época

Status
  - Tipo de datos: Cadena
  - Descripción: Indica si el evento ha sido revisado por un humano.

idss
  - Tipo de datos: Cadena
  - Descripción: una lista separada por comas de ID de eventos que están asociados a un evento.


In [32]:
df_sismo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   id                         20000 non-null  object        
 1   Magnitud                   20000 non-null  float64       
 2   Place                      19953 non-null  object        
 3   Primer Registro            20000 non-null  datetime64[ns]
 4   Ultimo registro            20000 non-null  datetime64[ns]
 5   Felt                       9872 non-null   float64       
 6   cdi                        9872 non-null   float64       
 7   mmi                        2760 non-null   float64       
 8   Posibilidad tsunami        20000 non-null  int64         
 9   Importancia del evento     20000 non-null  int64         
 10  ids                        20000 non-null  object        
 11  nst                        14445 non-null  float64       
 12  Dist

In [33]:
df_sismo.isnull().sum()

id                               0
Magnitud                         0
Place                           47
Primer Registro                  0
Ultimo registro                  0
Felt                         10128
cdi                          10128
mmi                          17240
Posibilidad tsunami              0
Importancia del evento           0
ids                              0
nst                           5555
Dist Horizontal epicentro      891
RMS                              2
Brecha azimutal                 18
estado                         384
Longitud                         0
Latitud                          0
Profundidad                      0
dtype: int64

In [113]:
#folder_path = "../DASHBOARD/CSV_TRANSFORMADOS"
#file_name = "Datos_USA.csv"
#file_path = os.path.join(folder_path, file_name)
#
#df_sismo.to_csv(file_path, index=False)